In [ ]:
import ast
import pandas as pd

In [ ]:
df = pd.read_csv("./outputs/gbh_output_all_fields.csv")

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=[
    'Unnamed: 0',
    '_id',
    'neighborhoods[0]',
    'neighborhoods[1]',
    'neighborhoods[2]',
    'neighborhoods[3]',
    'neighborhoods[4]',
    'neighborhoods[5]',
    'tracts[0]', 
    'tracts[1]',
    'tracts[2]', 
    'tracts[3]', 
    'tracts[4]', 
    'tracts[5]', 
    'tracts[6]',
    'hl1_y', 
    'body_y',
    'llama_prediction',
    'Explicit_Pass_1', 
    'NER_Pass_1', 
    'NER_Pass_1_Sorted',
    'NER_Pass_1_Coordinates', 
    'NER_prediction', 
    'NER_Sorted',
    'NER_Sorted_Coordinates',
    'topic_model_body', 
    'tokens',
    'ada_embedding', 
    'closest_topic_all', 
    'closest_topic_selected',
    'link.$regexReplace.replacement',
    'link.$regexReplace.input',
    'link.$regexReplace.find',
    'openai_labels[0]'
])

In [ ]:
df = df.rename(columns={"Tracts": "tracts", 'hl1_x': 'hl1','body_x': 'body', 'closest_topic_client': 'openai_labels'})

In [ ]:
df.dtypes

In [ ]:
neigh_tract_dict = {
    "Fenway" : ["010103", "010104", "010204", "010408", "010404", "010403", "981501", "010405", "010206", "010205"],
	"Downtown": ["030302", "070202", "070102", "030301", "070104", "070103", "070201"],
	"Beacon Hill": ["020200", "020302", "020101", "981700"],
	"Dorchester" : [ "092400", "091400", "090300", "091800", "092300", "100601", "090901", "100400", "090100", "091001","090200", "100200", "091700", "092200", "090700", "091500", "091300", "100300", "100100", "092000", "100500", "100800", "100603", "091200", "100700", "092101", "091900", "091600", "091100"],
	"Mattapan": ["100900", "101002", "101102", "981100", "101001","101101"],
	"Jamaica Plain": ["120103", "981800", "110105", "120600", "120700", "120301", "081200", "120105","081101", "981000", "120500", "120104", "120201", "110106", "081301", "120400"],
	"Roslindale": ["110502", "110104", "110501", "110401", "140106", "110301", "110607", "110403","110201"],
	"Roxbury": ["081500", "080500", "070801", "080100", "081800", "980300", "082000", "080601", "081700", "080300", "090600", "081400", "090400", "070901", "082100", "081900", "081302","080401"],
	"West End": ["020304", "020301", "020305"],
	"Longwood": ["010300", "081001"],
	"South Boston": ["061101", "060700", "060101", "061201", "061000", "060800", "981201", "060200", "061202", "060400", "061203", "060301", "060601", "060501"],
	"Back Bay": ["010702", "010701", "010802", "010801", "010500", "010600"],
	"Charlestown": ["040100", "040300", "040401", "040600", "040801", "040200"],
	"Allston": ["000604", "000804", "000703", "000704", "000806", "000101", "000807", "000701", "000805"],
	"Hyde Park": ["140107", "140201", "140105", "980700", "140300", "140202", "140400", "140102"],
	"East Boston": ["050500", "050600", "981502", "050101", "981300", "050901", "050300", "050700", "050400", "051000", "981600", "051200", "050200", "051101"],
	"South End": ["070301", "070302", "070502", "070501", "071101", "070600", "070700", "070902", "070802", "071201", "070402"],
	"West Roxbury": ["980900", "130406", "981900", "130404", "110601", "130300", "130402", "130200", "130101"],
	"South Boston Waterfront": ["981202", "060602", "060603", "061204", "060604"],
    "North End": ["030200", "030100", "030500", "030400"]
}

In [ ]:
def lookUp(tract):
    for key in neigh_tract_dict.keys():
        if (ast.literal_eval(tract)[0] in neigh_tract_dict[key]):
            return [key]
    return ["Greater Boston"]

In [ ]:
df['neighborhoods'] = df["tracts"].apply(lookUp)

In [ ]:
arrify = lambda x: [x]

In [ ]:
df['openai_labels'] = df["openai_labels"].apply(arrify)

In [ ]:
gbh_df = df[df['userID'] == 'org_2bHDzl2Zax0nILIzDhui2DLWdH6']

In [ ]:
df.to_csv("./prod_data.csv")

In [ ]:
df['userID'].value_counts()

### Reading Each Year

In [ ]:
import ast
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
year = 2010
csvs = [
    f'./combination/Boston_Globe_{year}_1.csv', 
    f'./combination/Boston_Globe_{year}_2.csv', 
    f'./combination/Boston_Globe_{year}_3.csv', 
    f'./combination/Boston_Globe_{year}_4.csv', 
    f'./combination/Boston_Globe_{year}_5.csv',
]
merged_df = pd.concat([pd.read_csv(file) for file in csvs])

In [ ]:
merged_df.columns

In [ ]:
merged_df = merged_df.drop(columns=['Unnamed: 0'])

In [ ]:
merged_df

### Merging all the CSVs for Boston Globe

In [ ]:
import re
from bs4 import BeautifulSoup

In [ ]:
master_df = pd.read_csv("./sample_data/cleaned_articles_mass.csv")

In [ ]:
func_clean_html = lambda x: BeautifulSoup(x, "html.parser").get_text()
master_df['body'] = master_df['body'].progress_apply(func_clean_html)
master_df['hl1'] = master_df['hl1'].progress_apply(func_clean_html)

func_clean_regex = lambda x: ' '.join([item for item in re.findall(r'[A-Za-z0-9!@#$%^&*().]+', x) if len(item) > 1])
master_df['body'] = master_df['body'].progress_apply(func_clean_regex)
master_df['hl1'] = master_df['hl1'].progress_apply(func_clean_regex)

### Combine

In [ ]:
final_df = pd.merge(merged_df, master_df, on='body', how='inner')

In [ ]:
final_df

In [ ]:
final_df.to_csv(f"./outputs/Boston_Globe_{year}_5000_articles_approx.csv")

### Final Combine

In [ ]:
arr = [
    f'./combination/Boston_Globe_2010_5000_articles_approx.csv', 
    f'./combination/Boston_Globe_2011_5000_articles_approx.csv', 
    f'./combination/Boston_Globe_2012_5000_articles_approx.csv', 
    f'./combination/Boston_Globe_2013_5000_articles_approx.csv', 
    f'./combination/Boston_Globe_2014_5000_articles_approx.csv',
    f'./combination/Boston_Globe_2015_5000_articles_approx.csv',
    f'./combination/Boston_Globe_2016_5000_articles_approx.csv',
    f'./combination/Boston_Globe_2017_5000_articles_approx.csv',
    f'./combination/Boston_Globe_2018_5000_articles_approx.csv',
    f'./combination/Boston_Globe_2019_5000_articles_approx.csv',
]

In [ ]:
final_final_df = pd.concat([pd.read_csv(file) for file in arr])

In [ ]:
final_final_df = final_final_df.drop(columns=['Unnamed: 0'])

In [ ]:
final_final_df.to_csv("./outputs/final_df_half.csv")

#### Quick File

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./sample_data/se_naacp_db.articles_data.csv")

In [3]:
df.columns

Index(['_id', 'neighborhoods[0]', 'neighborhoods[1]', 'neighborhoods[2]',
       'neighborhoods[3]', 'neighborhoods[4]', 'neighborhoods[5]',
       'position_section', 'tracts[0]', 'tracts[1]', 'tracts[2]', 'tracts[3]',
       'tracts[4]', 'tracts[5]', 'tracts[6]', 'author', 'body', 'content_id',
       'hl1', 'hl2', 'pub_date', 'pub_name', 'link', 'openai_labels[0]',
       'userID', 'uploadID', 'dateSum', 'link.$regexReplace.input',
       'link.$regexReplace.find', 'link.$regexReplace.replacement'],
      dtype='object')

In [4]:
df = df.drop(columns=['_id', 'neighborhoods[0]', 'neighborhoods[1]', 'neighborhoods[2]',
       'neighborhoods[3]', 'neighborhoods[4]', 'neighborhoods[5]',
       'position_section', 'tracts[0]', 'tracts[1]','content_id', 'tracts[2]', 'tracts[3]',
       'tracts[4]', 'tracts[5]', 'tracts[6]','hl2','openai_labels[0]',
       'userID', 'uploadID', 'dateSum', 'link.$regexReplace.input',
       'link.$regexReplace.find', 'link.$regexReplace.replacement'])

In [5]:
df.columns

Index(['author', 'body', 'hl1', 'pub_date', 'pub_name', 'link'], dtype='object')

In [6]:
df = df.rename(columns={'author': 'Byline', 'body':'Body', 'hl1':'Headline', 'pub_date':'Publish Date', 'pub_name':'Publisher', 'link':'Paths'})

In [7]:
df = df[:10]

In [8]:
df.to_csv("./outputs/test_2.csv", index=False)